## Error Detection

### Importing

In [18]:
import polars as pl
import pandas as pd
import matplotlib.pyplot as plt

### Data File

In [19]:
df = pl.read_csv(
    "example_trigger_data_large.csv",
    schema_overrides={
        "trigger_data": pl.UInt32,
        "trigger_clk": pl.UInt64,
        "trigger_id": pl.UInt16,
        "veto_in": pl.Boolean,
        "internal_trigger": pl.Boolean,
    }
)
print(df)

shape: (1_000_000, 5)
┌──────────────┬──────────────────────┬────────────┬─────────┬──────────────────┐
│ trigger_data ┆ trigger_clk          ┆ trigger_id ┆ veto_in ┆ internal_trigger │
│ ---          ┆ ---                  ┆ ---        ┆ ---     ┆ ---              │
│ u32          ┆ u64                  ┆ u16        ┆ bool    ┆ bool             │
╞══════════════╪══════════════════════╪════════════╪═════════╪══════════════════╡
│ 2387946004   ┆ 5129295877036350564  ┆ 36744      ┆ false   ┆ false            │
│ 3705039394   ┆ 5896637771063441370  ┆ 31836      ┆ false   ┆ false            │
│ 4135854240   ┆ 15909143681047300436 ┆ 37080      ┆ false   ┆ false            │
│ 2966489412   ┆ 4283887664210683987  ┆ 18203      ┆ true    ┆ false            │
│ 1369896137   ┆ 18412677095183464827 ┆ 49083      ┆ false   ┆ false            │
│ …            ┆ …                    ┆ …          ┆ …       ┆ …                │
│ 3460009179   ┆ 18090895291116405222 ┆ 49575      ┆ false   ┆ true         

### Time Conversion

In [20]:
df = df.with_columns(
    (pl.col("trigger_clk") / 40000000).alias("trigger_clk")
)

#### Printing DataFrame

In [21]:
print(df)

shape: (1_000_000, 5)
┌──────────────┬─────────────┬────────────┬─────────┬──────────────────┐
│ trigger_data ┆ trigger_clk ┆ trigger_id ┆ veto_in ┆ internal_trigger │
│ ---          ┆ ---         ┆ ---        ┆ ---     ┆ ---              │
│ u32          ┆ f64         ┆ u16        ┆ bool    ┆ bool             │
╞══════════════╪═════════════╪════════════╪═════════╪══════════════════╡
│ 2387946004   ┆ 1.2823e11   ┆ 36744      ┆ false   ┆ false            │
│ 3705039394   ┆ 1.4742e11   ┆ 31836      ┆ false   ┆ false            │
│ 4135854240   ┆ 3.9773e11   ┆ 37080      ┆ false   ┆ false            │
│ 2966489412   ┆ 1.0710e11   ┆ 18203      ┆ true    ┆ false            │
│ 1369896137   ┆ 4.6032e11   ┆ 49083      ┆ false   ┆ false            │
│ …            ┆ …           ┆ …          ┆ …       ┆ …                │
│ 3460009179   ┆ 4.5227e11   ┆ 49575      ┆ false   ┆ true             │
│ 1328686175   ┆ 1.8433e11   ┆ 1575       ┆ true    ┆ false            │
│ 3564652371   ┆ 4.1819e11   

### Trigger Data Errors

In [22]:
td_df = df.with_columns(
    pl.col("trigger_data").cast(pl.UInt32).map_elements(
        lambda x: format(x, '032b'),
        return_dtype=pl.String
    ).alias("trigger_data_binary")
)

for i in range(12):
    start = 8 + i * 2
    end = start + 2
    td_df = td_df.with_columns(
        td_df["trigger_data_binary"].str.slice(start, 2).map_elements(
            lambda x: int(x, 2),
            return_dtype=pl.UInt8
        ).alias(f"chunk_{i}")
    )

td_df = td_df.filter(
    pl.struct(td_df.columns).map_elements(lambda row: 0 not in row and 3 not in row)
)
    
td_df = td_df.drop("trigger_data_binary").drop("trigger_data")

msb_df = td_df.select(
    [f"chunk_{i}" for i in range(6)]
)

lsb_df = td_df.select(
    [f"chunk_{i}" for i in range(6, 12)]
)

#### Printing DataFrames

In [23]:
print(msb_df)

shape: (1_000_000, 6)
┌─────────┬─────────┬─────────┬─────────┬─────────┬─────────┐
│ chunk_0 ┆ chunk_1 ┆ chunk_2 ┆ chunk_3 ┆ chunk_4 ┆ chunk_5 │
│ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---     │
│ u8      ┆ u8      ┆ u8      ┆ u8      ┆ u8      ┆ u8      │
╞═════════╪═════════╪═════════╪═════════╪═════════╪═════════╡
│ 1       ┆ 1       ┆ 1       ┆ 1       ┆ 0       ┆ 2       │
│ 3       ┆ 1       ┆ 1       ┆ 2       ┆ 1       ┆ 2       │
│ 2       ┆ 0       ┆ 1       ┆ 0       ┆ 0       ┆ 2       │
│ 3       ┆ 1       ┆ 0       ┆ 1       ┆ 0       ┆ 0       │
│ 2       ┆ 2       ┆ 1       ┆ 2       ┆ 3       ┆ 3       │
│ …       ┆ …       ┆ …       ┆ …       ┆ …       ┆ …       │
│ 0       ┆ 3       ┆ 2       ┆ 3       ┆ 2       ┆ 0       │
│ 0       ┆ 3       ┆ 0       ┆ 2       ┆ 0       ┆ 2       │
│ 1       ┆ 3       ┆ 2       ┆ 0       ┆ 1       ┆ 0       │
│ 1       ┆ 3       ┆ 2       ┆ 1       ┆ 0       ┆ 1       │
│ 2       ┆ 3       ┆ 3       ┆ 3       ┆ 3     

In [24]:
print(lsb_df)

shape: (1_000_000, 6)
┌─────────┬─────────┬─────────┬─────────┬──────────┬──────────┐
│ chunk_6 ┆ chunk_7 ┆ chunk_8 ┆ chunk_9 ┆ chunk_10 ┆ chunk_11 │
│ ---     ┆ ---     ┆ ---     ┆ ---     ┆ ---      ┆ ---      │
│ u8      ┆ u8      ┆ u8      ┆ u8      ┆ u8       ┆ u8       │
╞═════════╪═════════╪═════════╪═════════╪══════════╪══════════╡
│ 2       ┆ 2       ┆ 0       ┆ 1       ┆ 1        ┆ 0        │
│ 2       ┆ 2       ┆ 0       ┆ 2       ┆ 0        ┆ 2        │
│ 0       ┆ 0       ┆ 2       ┆ 2       ┆ 0        ┆ 0        │
│ 2       ┆ 1       ┆ 1       ┆ 0       ┆ 1        ┆ 0        │
│ 1       ┆ 0       ┆ 3       ┆ 0       ┆ 2        ┆ 1        │
│ …       ┆ …       ┆ …       ┆ …       ┆ …        ┆ …        │
│ 3       ┆ 0       ┆ 3       ┆ 1       ┆ 2        ┆ 3        │
│ 1       ┆ 0       ┆ 1       ┆ 1       ┆ 3        ┆ 3        │
│ 1       ┆ 3       ┆ 1       ┆ 1       ┆ 0        ┆ 3        │
│ 0       ┆ 3       ┆ 0       ┆ 1       ┆ 1        ┆ 0        │
│ 2       ┆ 2     

### Veto In Errors

In [25]:
vi_df = df.filter(pl.col("veto_in") == True)

#### Printing DataFrames

In [26]:
print(vi_df)

shape: (499_746, 5)
┌──────────────┬─────────────┬────────────┬─────────┬──────────────────┐
│ trigger_data ┆ trigger_clk ┆ trigger_id ┆ veto_in ┆ internal_trigger │
│ ---          ┆ ---         ┆ ---        ┆ ---     ┆ ---              │
│ u32          ┆ f64         ┆ u16        ┆ bool    ┆ bool             │
╞══════════════╪═════════════╪════════════╪═════════╪══════════════════╡
│ 2966489412   ┆ 1.0710e11   ┆ 18203      ┆ true    ┆ false            │
│ 2379684554   ┆ 5.6787e10   ┆ 26210      ┆ true    ┆ true             │
│ 1345569140   ┆ 1.6859e11   ┆ 13171      ┆ true    ┆ false            │
│ 2598166348   ┆ 3.4165e10   ┆ 2082       ┆ true    ┆ true             │
│ 1068335650   ┆ 9.9311e10   ┆ 45718      ┆ true    ┆ true             │
│ …            ┆ …           ┆ …          ┆ …       ┆ …                │
│ 2423129721   ┆ 2.0816e10   ┆ 19562      ┆ true    ┆ false            │
│ 1328686175   ┆ 1.8433e11   ┆ 1575       ┆ true    ┆ false            │
│ 3564652371   ┆ 4.1819e11   ┆ 

### Internal Trigger Errors

In [27]:
it_df = df.filter(pl.col("internal_trigger") == True)

#### Printing DataFrames

In [28]:
print(it_df)

shape: (500_995, 5)
┌──────────────┬─────────────┬────────────┬─────────┬──────────────────┐
│ trigger_data ┆ trigger_clk ┆ trigger_id ┆ veto_in ┆ internal_trigger │
│ ---          ┆ ---         ┆ ---        ┆ ---     ┆ ---              │
│ u32          ┆ f64         ┆ u16        ┆ bool    ┆ bool             │
╞══════════════╪═════════════╪════════════╪═════════╪══════════════════╡
│ 2379684554   ┆ 5.6787e10   ┆ 26210      ┆ true    ┆ true             │
│ 1832620821   ┆ 5.2879e10   ┆ 56027      ┆ false   ┆ true             │
│ 151272243    ┆ 1.9475e11   ┆ 56791      ┆ false   ┆ true             │
│ 2598166348   ┆ 3.4165e10   ┆ 2082       ┆ true    ┆ true             │
│ 1068335650   ┆ 9.9311e10   ┆ 45718      ┆ true    ┆ true             │
│ …            ┆ …           ┆ …          ┆ …       ┆ …                │
│ 2976640140   ┆ 2.4010e11   ┆ 20350      ┆ true    ┆ true             │
│ 1611905553   ┆ 4.5765e11   ┆ 58231      ┆ false   ┆ true             │
│ 1060942953   ┆ 3.7210e11   ┆ 

## Plotting

In [29]:
'''
-x = clk, y = faulty ids (tells us if there is an interval which issues happen)
-heat map (graph that changes color when get 0 or 2)
-3d gaussian distribution x = , y = , z = (tells us)
'''

'\n-x = clk, y = faulty ids (tells us if there is an interval which issues happen)\n-heat map (graph that changes color when get 0 or 2)\n-3d gaussian distribution x = , y = , z = (tells us)\n'

### Faulty Rods at Intervals

### Heat Map

In [30]:
data = pd.read_csv ("example_trigger_data_large.csv")

hit_count = df["trigger_clk"]
x_labels = hit_count
y_labels = ["0/1","2/3", "4/5", "6/7", "8/9","10/11", "12/13", "14/15",
            "16/17", "18/19", "20/21", "22/23"]
fig = px.imshow(data,
                labels = dict(x = "Hits", y = "Channels"), color = "viridis",),
x =x_labels, y = y_labels # what is data supposed to be?

fig.update_xaxes(side="bottom") #not sure that it's called bottom, could be called bot
fig.show()

NameError: name 'px' is not defined

### Gaussian Distribution